#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [188]:
import pandas as pd
import pymongo
import datetime

In [189]:
client = pymongo.MongoClient('mongo')
database = client['exercices']
collection = database['kickstarter']

In [190]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [191]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

In [192]:
len(df_ks)

150000

## Question 0

### Netoyer les données

In [193]:
df_ks = df_ks.rename(columns={"ID" : "_id"})
df_ks['launched'] = pd.to_datetime(df_ks['launched'], format='%Y-%m-%d', errors = "coerce")
df_ks['deadline'] = pd.to_datetime(df_ks['deadline'], format='%Y-%m-%d', errors = "coerce")
df_ks['backers'] = pd.to_numeric(df_ks['backers'], errors="coerce")
df_ks['goal'] = pd.to_numeric(df_ks['goal'], errors="coerce")
df_ks['pledged'] = pd.to_numeric(df_ks['pledged'], errors="coerce")
df_ks['usd pledged'] = pd.to_numeric(df_ks['usd pledged'], errors="coerce")

df_ks = df_ks.dropna()

df_ks['backers'] = df_ks['backers'].astype("int")
df_ks['goal'] = df_ks['goal'].astype("int")
df_ks['pledged'] = df_ks['pledged'].astype("int")
df_ks['usd pledged'] = df_ks['usd pledged'].astype("float")



In [194]:
len(df_ks)

148515

### Importer les données

In [195]:
collection.delete_many({})
collection.insert_many(df_ks.to_dict(orient='records'))
cursor = collection.find()

## Question 1  

In [196]:
most_donate = collection.aggregate([
    {"$sort":{"usd_pledged_real":-1}},
    {"$limit" : 5}
])
list(most_donate)

[{'_id': 342886736,
  'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler",
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': datetime.datetime(2014, 8, 30, 0, 0),
  'goal': 50000,
  'launched': datetime.datetime(2014, 7, 8, 10, 14, 37),
  'pledged': 13285226,
  'state': 'successful',
  'backers': 62642,
  'country': 'US',
  'usd pledged': 13285226.36,
  'usd_pledged_real': 13285226.36},
 {'_id': 2103598555,
  'name': 'Pebble 2, Time 2 + All-New Pebble Core',
  'category': 'Product Design',
  'main_category': 'Design',
  'currency': 'USD',
  'deadline': datetime.datetime(2016, 6, 30, 0, 0),
  'goal': 1000000,
  'launched': datetime.datetime(2016, 5, 24, 15, 49, 52),
  'pledged': 12779843,
  'state': 'successful',
  'backers': 66673,
  'country': 'US',
  'usd pledged': 12779843.49,
  'usd_pledged_real': 12779843.49},
 {'_id': 1033978702,
  'name': 'OUYA: A New Kind of Video Game Console',
  'category': 'Gaming Hardware',
  'm

## Question 2

In [197]:
number_state_ok = collection.find({"state":"successful"})
len(list(number_state_ok))

52998

## Question 3

In [198]:
number_project = collection.aggregate([{"$group" : {"_id" : "$category", "numberprojectbycategory" : {"$sum" : 1}}}])
list(number_project)

[{'_id': 'Gaming Hardware', 'numberprojectbycategory': 178},
 {'_id': 'Periodicals', 'numberprojectbycategory': 517},
 {'_id': 'Performance Art', 'numberprojectbycategory': 855},
 {'_id': 'Translations', 'numberprojectbycategory': 76},
 {'_id': 'Academic', 'numberprojectbycategory': 367},
 {'_id': 'Wearables', 'numberprojectbycategory': 508},
 {'_id': 'Comics', 'numberprojectbycategory': 1931},
 {'_id': "Farmer's Markets", 'numberprojectbycategory': 175},
 {'_id': 'Animals', 'numberprojectbycategory': 97},
 {'_id': 'Video Games', 'numberprojectbycategory': 4797},
 {'_id': 'Thrillers', 'numberprojectbycategory': 302},
 {'_id': 'Action', 'numberprojectbycategory': 282},
 {'_id': 'Food', 'numberprojectbycategory': 4612},
 {'_id': 'Interactive Design', 'numberprojectbycategory': 146},
 {'_id': 'Calendars', 'numberprojectbycategory': 112},
 {'_id': 'Country & Folk', 'numberprojectbycategory': 1790},
 {'_id': 'Letterpress', 'numberprojectbycategory': 24},
 {'_id': 'Indie Rock', 'numberprojec

## Question 4

In [199]:
number_french_project_b2016 = collection.find({"$and" : [ {"country":"FR"}, {"launched": {"$lt" : datetime.datetime(2016, 1, 1, 0, 0, 0, 0)}}]})
list(number_french_project_b2016)

[{'_id': 545482362,
  'name': 'Depeche Mode Bar Belgium',
  'category': 'Pop',
  'main_category': 'Music',
  'currency': 'EUR',
  'deadline': datetime.datetime(2015, 7, 1, 0, 0),
  'goal': 20000,
  'launched': datetime.datetime(2015, 5, 27, 11, 29, 28),
  'pledged': 305,
  'state': 'failed',
  'backers': 10,
  'country': 'FR',
  'usd pledged': 333.98121895,
  'usd_pledged_real': 338.42637284599937},
 {'_id': 400288569,
  'name': 'Dracucat : A vampire kitten card game',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'currency': 'EUR',
  'deadline': datetime.datetime(2015, 10, 29, 0, 0),
  'goal': 1900,
  'launched': datetime.datetime(2015, 9, 29, 7, 4, 8),
  'pledged': 2944,
  'state': 'successful',
  'backers': 129,
  'country': 'FR',
  'usd pledged': 3295.37237632,
  'usd_pledged_real': 3243.3980764357875},
 {'_id': 1990332786,
  'name': 'Minotaur enjoy your life & your ice cream',
  'category': 'Food Trucks',
  'main_category': 'Food',
  'currency': 'EUR',
  'deadline'

## Question 5

In [200]:
number_usa_project = collection.find({"$and" : [ {"country":"US"}, {"goal": {"$gte" : 200000}}]})
list(number_usa_project)

[{'_id': 655043686,
  'name': 'Far from Par is a movie about a man and a talking golf ball.',
  'category': 'Comedy',
  'main_category': 'Film & Video',
  'currency': 'USD',
  'deadline': datetime.datetime(2014, 12, 5, 0, 0),
  'goal': 200000,
  'launched': datetime.datetime(2014, 10, 6, 21, 20, 6),
  'pledged': 10,
  'state': 'failed',
  'backers': 2,
  'country': 'US',
  'usd pledged': 10.0,
  'usd_pledged_real': 10.0},
 {'_id': 866634482,
  'name': 'A CALL TO ADVENTURE',
  'category': 'Film & Video',
  'main_category': 'Film & Video',
  'currency': 'USD',
  'deadline': datetime.datetime(2012, 9, 14, 0, 0),
  'goal': 287000,
  'launched': datetime.datetime(2012, 8, 13, 23, 14, 2),
  'pledged': 1465,
  'state': 'failed',
  'backers': 11,
  'country': 'US',
  'usd pledged': 1465.0,
  'usd_pledged_real': 1465.0},
 {'_id': 993194166,
  'name': 'Storybricks, the storytelling online RPG',
  'category': 'Video Games',
  'main_category': 'Games',
  'currency': 'USD',
  'deadline': datetime.d

## Question 6 

In [186]:
collection.createIndex({"name":"text"})
name_sport = collection.find( { "$text": { "$search": "sport" } } )

TypeError: 'Collection' object is not callable. If you meant to call the 'createIndex' method on a 'Collection' object it is failing because no such method exists.